In [ ]:
import numpy as np
from scipy.integrate import tplquad 
from jupyprint import jupyprint, arraytex
from numdifftools import Hessian, Jacobian

In [ ]:
# reference
# constant Z
constant_Z = tplquad(
    lambda x, y, z: np.exp(-np.linalg.norm(np.array([x, y, z]))**4/4), 
    -np.inf, np.inf,
    -np.inf, np.inf,
    -np.inf, np.inf)[0]

In [ ]:
# reference
# constant Z
EX2 = tplquad(
    lambda x, y, z: np.linalg.norm(np.array([x, y, z])) ** 2 \
        * np.exp(-np.linalg.norm(np.array([x, y, z]))**4/4) / constant_Z, 
    -np.inf, np.inf,
    -np.inf, np.inf,
    -np.inf, np.inf
    )[0]

EX4 = tplquad(
    lambda x, y, z: np.linalg.norm(np.array([x, y, z])) ** 4 \
        * np.exp(-np.linalg.norm(np.array([x, y, z]))**4/4) / constant_Z, 
    -np.inf, np.inf,
    -np.inf, np.inf,
    -np.inf, np.inf
    )[0]

EX6 = tplquad(
    lambda x, y, z: np.linalg.norm(np.array([x, y, z])) ** 6 \
        * np.exp(-np.linalg.norm(np.array([x, y, z]))**4/4) / constant_Z, 
    -np.inf, np.inf,
    -np.inf, np.inf,
    -np.inf, np.inf
    )[0]

In [ ]:
# define stationary distribution 
def density(x:float) -> float:
    return np.exp(-potential(x))/constant_Z

In [ ]:
# test of gradient 
h = 1e-5
x0 = np.array([0,-1,3])

jupyprint("$\\hat{f}(x_0) = "+str(arraytex(Jacobian(potential)(x0)))+"$")
jupyprint("$f(x_0) = "+str(arraytex(gradient(x0)))+"$")
jupyprint("$\\hat{f}^2(x_0) = "+str(arraytex(Hessian(potential)(x0)))+"$")
jupyprint("$f^2(x_0) = "+str(arraytex(hessian(x0)))+"$")

In [ ]:
jupyprint(f"$Z = {constant_Z}$")
jupyprint("$\\mathbb{E} |X|^2 = "+ str(EX2)+"$" )
jupyprint("$\\mathbb{E} |X|^4 = "+ str(EX4)+"$" )
jupyprint("$\\mathbb{E} |X|^6 = "+ str(EX6)+"$" )

In [ ]:
# TRASH BIN FOR HAMILTONIAN MCMC

# Hamiltonian Monte Carlo
M = 1
L = 200
epsilon = 0.1

# assign Hamiltonian
hamiltonian = lambda p,q: potential(p) + 1/2 * q.T.dot(np.linalg.inv(M)).dot(q)


for i_MC in range(0, n_MC):
    clear_output(wait=True)
    print("MC: "+str(i_MC+1)+"/"+str(n_MC))

    samples_hamiltonian = [initial_sample]

    accept_count = 0

    for _ in tqdm(range(1, number_of_samples)):

        # get proposal -> Hamiltonian mechanics
        # draw momnentum q
        q0 = sp.norm.rvs(loc = 0,scale=M, size=d)
        #assign position
        p0 = samples_hamiltonian[-1]


        p = p0
        q = q0
        # Leagprof algorithm
        for i_L in range(0,L):
            q = q - epsilon/2 * gradient(p)
            p = p + epsilon * q
            q = q - epsilon/2 * gradient(p)
        
        if hamiltonian(p0,q0) == 0:
            alpha = 1
        else:
            alpha = np.min([
                1,
                np.exp(-hamiltonian(p,q))/np.exp(-hamiltonian(p0,q0))
                ])

        u = sp.uniform.rvs()

        if u <= alpha:
            samples_hamiltonian.append(p)
            samples_hamiltonian_norm.append(np.linalg.norm(p))
            accept_count += 1
        else:
            samples_hamiltonian.append(p0)
            samples_hamiltonian_norm.append(samples_hamiltonian_norm[-1])

    jupyprint("acceptance rate = $" + str(np.round(accept_count/number_of_samples))+ "$")

    samples_hamiltonian = np.array(samples_hamiltonian[tune_interval:])
    samples_hamiltonian_norm = np.array(samples_hamiltonian_norm[tune_interval:])

    moment2_hamiltonian.append(np.mean(samples_hamiltonian_norm**2))
    moment4_hamiltonian.append(np.mean(samples_hamiltonian_norm**4))
    moment6_hamiltonian.append(np.mean(samples_hamiltonian_norm**6))